In [2]:
import pyodbc
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
#connection details (server, database, username, password)
server = '172.10.2.5'
database = 'Car Store DB'
username = 'Carstore'
password = 'nss@12345'
class TextColors:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    RESET = '\033[0m'
    
def fetch_car_options(connection):
    query = 'SELECT carid, car_company, car_name, car_model, car_mileage, car_color, car_price,stock FROM car_options'
    car_options_df = pd.read_sql(query, connection)
    return car_options_df
def fetch_bank_details(connection):
    query = 'SELECT bank_id, bank_name, bank_interest FROM Bank_Details'
    bank_details_df = pd.read_sql(query, connection)
    return bank_details_df
def fetch_user_id(connection):
    try:
        cursor = connection.cursor()
        # Fetch the user id and name of user based on the latest login timestamp
        cursor.execute("SELECT TOP 1 u_id, fullname FROM users ORDER BY last_login_timestamp DESC")
        row = cursor.fetchone()
        if row:
            u_id, fullname = row.u_id, row.fullname
            return u_id, fullname
        else:
            print("No user found with login history.")
            return None, None
    except Exception as e:
        print(f'Failed to fetch data: {str(e)}')
        return None, None
    
def generate_car_invoice_df(selected_car, selected_bank, purchase_timestamp):
    invoice_data = {
        "Car Company": [selected_car['car_company'].values[0]],
        "Car Name": [selected_car['car_name'].values[0]],
        "Car Model": [selected_car['car_model'].values[0]],
        "Car Color": [selected_car['car_color'].values[0]],
        "Car Price (Rs.)": [selected_car['car_price'].values[0]],
        "Bank Name" : [selected_bank['bank_name'].values[0]]
    }
  
    invoice_df = pd.DataFrame(invoice_data)
    return invoice_df
 
def select_car_and_payment(connection):
    car_options_df = fetch_car_options(connection)
    bank_details_df = fetch_bank_details(connection)
    exit_flag = False 
    while not exit_flag:
        try:
            # Display car options
            car_opt = pd.DataFrame(car_options_df)
            text5 = "-Car Options-"
            formatted_text = f"\033[1m{text5.center(80)}\033[0m"
            print(formatted_text)
            car_opt.reset_index(drop=True, inplace=True)
 
            print("\n",car_opt)
            while True:
                selected_carid = int(input("Enter the carid of the car you want to choose: "))
                selected_car = car_options_df[car_options_df['carid'] == selected_carid]
                if not selected_car.empty:
                    break
                else:
                    print(TextColors.RED+"Car with the specified carid not found. Please enter a valid carid."+TextColors.RESET)
            # Display the selected car
            print(f"\033[1mYou have selected the following car :\033[0m")
            print(f"Car ID: {selected_car['carid'].values[0]}")
            print(f"Car Company: {selected_car['car_company'].values[0]}")
            print(f"Car Name: {selected_car['car_name'].values[0]}")
            print(f"Car Model: {selected_car['car_model'].values[0]}")
            print(f"Car Mileage: {selected_car['car_mileage'].values[0]} MPG")
            print(f"Car Color: {selected_car['car_color'].values[0]}")
            print(f"Car Price: Rs.{selected_car['car_price'].values[0]:.2f}")
            # Ask for the payment option
            while True:
                payment_option = input("Choose a payment option 1.Full Payment or 2.EMI : ")
                if payment_option == "1" or payment_option == "2":
                    break
                else:
                    print(TextColors.RED+"Invalid payment option. Please choose either '1' for Full Payment or '2' for EMI."+TextColors.RESET)
            if payment_option == "1":
                bank_col = bank_details_df[['bank_id', 'bank_name']]
                while True:
                    # Display bank options for Full Payment
                    text6 = "-Bank Options For Full Payment-"
                    formatted_text = f"\033[1m{text6.center(80)}\033[0m"
                    print(formatted_text)
                    print(bank_col)
                    selected_bank_id = int(input("Enter the bank_id of the bank you want to choose for full payment: "))
                    selected_bank = bank_details_df[bank_details_df['bank_id'] == selected_bank_id]
                    if not selected_bank.empty:
                        break
                    else:
                        print(Textcolors.RED+"Bank with the specified bank_id not found. Please enter a valid bank_id for full payment."+Textcolors.RESET)
                print(f"\n\033[1mYou have selected the following bank for full payment :\033[0m")
                print(f"Bank ID: {selected_bank['bank_id'].values[0]}")
                print(f"Bank Name: {selected_bank['bank_name'].values[0]}")
                cursor = connection.cursor()
                purchase_timestamp = datetime.now()
                u_id, fullname = fetch_user_id(connection)
                if u_id is not None:
                    cursor.execute(
                                    "INSERT INTO car_selection (carid, car_name, car_price, bank_id, bank_name, bank_interest, purchase_timestamp, u_id, fullname,payment_method) VALUES (?,?, ?, ?, ?, ?, ?, ?, ?, ?)",
                                    int(selected_car['carid'].values[0]), selected_car['car_name'].values[0], float(selected_car['car_price'].values[0]),
                                    int(selected_bank['bank_id'].values[0]), selected_bank['bank_name'].values[0],
                                    0, purchase_timestamp, u_id, fullname,'Full')
                    connection.commit()
                    invoice_df = generate_car_invoice_df(selected_car, selected_bank, purchase_timestamp)
                    text7 = "-Invoice-"
                    formatted_text = f"\033[1m{text7.center(80)}\033[0m"
                    print(formatted_text)
                    print(invoice_df)
                    print("\n\n\nThank you for your purchase!")
                exit_flag = True
            elif payment_option == "2":
                bank_col = bank_details_df
                while True:
                    # Display bank options for EMI
                    print(f"\n\033[1m-Bank Options for EMI-\033[0m")
                    print(bank_col,"\n")
                    selected_bank_id = int(input("Enter the bank_id of the bank you want to choose for EMI"))
                    selected_bank = bank_details_df[bank_details_df['bank_id'] == selected_bank_id]
                    if not selected_bank.empty:   
                        break
                    else:
                        print(TextColors.RED+"Bank with the specified bank_id not found. Please enter a valid bank_id for EMI."+TextColors.RESET)
                print(f"\n\033[1mYou have selected the following bank for EMI\033[0m")
                print(f"\nBank ID: {selected_bank['bank_id'].values[0]}")
                cursor = connection.cursor()
                purchase_timestamp = datetime.now()
                u_id, fullname = fetch_user_id(connection)
                if u_id is not None:
                    cursor.execute(
                                "INSERT INTO car_selection (carid, car_name, car_price, bank_id, bank_name, bank_interest, purchase_timestamp, u_id, fullname,payment_method) VALUES ( ?,?, ?, ?, ?, ?, ?, ?, ?,?)",
                                (int(selected_car['carid'].values[0]), selected_car['car_name'].values[0], float(selected_car['car_price'].values[0]),
                                int(selected_bank['bank_id'].values[0]), selected_bank['bank_name'].values[0],
                                float(selected_bank['bank_interest'].values[0]), purchase_timestamp, u_id, fullname,'EMI'))
                    connection.commit()
                %run Emi.ipynb
                print("\n\n\nThank you for your purchase!")
                exit_flag = True

            exit_flag = True  
        except ValueError:
            print(textcolors.RED+"Invalid input. Please enter a valid value."+textcolors.RESET)
if __name__ == "__main__":
    connection = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};SERVER=" + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    select_car_and_payment(connection)

                                 -Car Options-                                  

    carid    car_company  car_name  car_model  car_mileage car_color  car_price
0      1         Toyota     Camry      sedan         30.5      Blue   150000.0
1      2          Honda     Civic      sedan         35.2       Red   200000.0
2      3           Ford  Explorer        Suv         23.8     Black   350000.0
3      4      Chevrolet     Cruze      sedan         28.1    Silver   180000.0
4      5           Ford  Ecosport        Suv         25.1     Black   180000.0
5      6  Maruti Suzuki     Swift  Hatchback         26.1     White   700000.0
Enter the carid of the car you want to choose: 3
You have selected the following car :
Car ID: 3
Car Company: Ford
Car Name: Explorer
Car Model: Suv
Car Mileage: 23.8 MPG
Car Color: Black
Car Price: Rs.350000.00
Choose a payment option 1.Full Payment or 2.EMI : 1
                        -Bank Options For Full Payment-                         
   bank_id         